In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

### Conv layer와 FC layer의 weight 수 비교

In [21]:
fc = nn.Linear(in_features=3 * 32 * 32, out_features=4)
cnn = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)

fc_num_weight = np.prod(fc.weight.size()) + fc.bias.size()[0]
cnn_num_weight = np.prod(cnn.weight.size()) + cnn.bias.size()[0]

print("fc layer weight :", fc_num_weight)
print("cnn layer weight :", cnn_num_weight)

fc layer weight : 12292
cnn layer weight : 1792


## block 쌓기

In [2]:
class ConvBlock(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, stride, padding):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=kernel_size,
                              stride=stride, padding=padding)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, inputs):
        x = self.conv(inputs)
        x = self.relu(x)
        x = self.pool(x)
        return x

In [31]:
conv = ConvBlock(3, 64, 3, 1, "same")
temp = torch.randn(1, 3, 32, 32)
conv(temp).size()

torch.Size([1, 64, 16, 16])

## 8.4 컨볼루션 신경망 구현

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_data = datasets.MNIST(root = './data/02/',
                            train=True,
                            download=True,
                            transform=transforms.ToTensor())
test_data = datasets.MNIST(root = './data/02/',
                            train=False,
                            download=True,
                            transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                           batch_size=128, shuffle=False)

class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = ConvBlock(1, 6, 5, 1, "same")
        self.conv2 = ConvBlock(6, 16, 5, 1, "valid")
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, padding="valid")
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.relu1 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(84, 10)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = torch.flatten(x, 1)
        x = self.relu1(self.fc1(x))
        out = self.fc2(x)
        return self.softmax(out)

    
def train_step(model, loader, optimzer, criterion):
    model.train()
    epoch_loss = 0
    for i, (data, target) in enumerate(loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        if (i + 1) % 100 == 0:
            print("Train Step : {}\tLoss : {:3f}".format(i + 1, loss.item()))
    return epoch_loss / len(loader)


def evaluate_step(model, loader, criterion):
    model.eval()
    epoch_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            loss = criterion(output, target)
            epoch_loss += loss.item()
            correct += output.argmax(1).eq(target).sum()
    return epoch_loss / len(loader), correct / len(loader.dataset)
    
    
model = LeNet().to(device)    
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(3):
    train_loss = train_step(model, train_loader, optimizer, criterion)
    test_loss, test_acc = evaluate_step(model, test_loader, criterion)
    print(f"EPOCH {epoch + 1:>3d} Train Loss {train_loss:>.3f} Test Loss {test_loss:>.3f} accuracy {test_acc:>3f}")

Train Step : 100	Loss : 1.863877
Train Step : 200	Loss : 1.792678
Train Step : 300	Loss : 1.806122
Train Step : 400	Loss : 1.789826
EPOCH   1 Train Loss 1.826 Test Loss 1.773 accuracy 0.685800
Train Step : 100	Loss : 1.756380
Train Step : 200	Loss : 1.791661
Train Step : 300	Loss : 1.673236
Train Step : 400	Loss : 1.701796
EPOCH   2 Train Loss 1.733 Test Loss 1.677 accuracy 0.781600
Train Step : 100	Loss : 1.583120
Train Step : 200	Loss : 1.607495
Train Step : 300	Loss : 1.572719
Train Step : 400	Loss : 1.596276
EPOCH   3 Train Loss 1.589 Test Loss 1.575 accuracy 0.884000


In [41]:
transform = transforms.Compose(
    [transforms.ToTensor()])

train_data = datasets.CIFAR10(root='./data/03', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data/03', train=False,
                             download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False)


class Cifar10_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv2 = self.build_layer(32, 32, 3, 1, "valid", 2, 0.25)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv4 = self.build_layer(64, 64, 3, 1, "valid", 2, 0.25)
        
        self.fc1 = nn.Linear(in_features=64 * 5 * 5, out_features=512)
        self.dr1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features=512, out_features=10)
        self.softmax = nn.Softmax(dim=1)
        
    def build_layer(self, in_channel, out_channel, kernel_size, stride, padding, pool_size, dropout):
        layer = nn.Sequential(
            nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=kernel_size, stride=stride,
                     padding=padding),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(pool_size),
            nn.Dropout(dropout)
        )
        return layer
    
    def forward(self, inputs):
        x = F.relu(self.conv1(inputs))
        x = self.conv2(x)
        x = F.relu(self.conv3(x))
        x = self.conv4(x)
        x = torch.flatten(x, 1)
        x = self.dr1(self.fc1(x))
        out = self.softmax(self.fc2(x))
        return out
    

model = Cifar10_CNN().to(device)    
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


for epoch in range(3):
    train_loss = train_step(model, train_loader, optimizer, criterion)
    test_loss, test_acc = evaluate_step(model, test_loader, criterion)
    print(f"EPOCH {epoch + 1:>3d} Train Loss {train_loss:>.3f} Test Loss {test_loss:>.3f} accuracy {test_acc:>3f}")

Files already downloaded and verified
Files already downloaded and verified
Train Step : 100	Loss : 2.166054
Train Step : 200	Loss : 2.148806
Train Step : 300	Loss : 2.148353
EPOCH   1 Train Loss 2.160 Test Loss 2.107 accuracy 0.344300
Train Step : 100	Loss : 2.090433


KeyboardInterrupt: 

## 8.6 우편번 인식기 v.2